In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import wrds 
import matplotlib.pyplot as plt

from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from scipy import stats

c:\Users\YeonChan Kang\anaconda3\envs\fdb\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Fama and MAcBeth (1973)   
control for a latge set of other variables.   
The relation of interest, as weel as the relation between each control variable the outcome variable of interest, is linear

In [2]:
conn = wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


1. 각 기간 t의 데이터를 사용, 하나 이상의 독립변수 X1, X2 등에 대해 관심있는 종속변수의 주기적인 cross-sectional regrsiion을 실시.   
각 기간의 각 독립변수에 대한 기울기 계수 및 절편계수가 생성   
2. 각 회귀계순의 시계열 분석, 평균계수가 0과 다른지 여부 확인

## 1. Periodic Cross-Sectional Regression

In [3]:
comp = conn.raw_sql("""
                    SELECT gvkey, datadate, at, pstkl, txditc, pstkrv, seq, pstk
                    FROM comp.funda
                    WHERE indfmt='INDL'
                    AND datafmt='STD'
                    AND popsrc='D'
                    AND consol='C'
                    AND datadate >= '01/01/1959'
                    """, date_cols=['datadate'])

In [4]:
comp

,gvkey,datadate,at,pstkl,txditc,pstkrv,seq,pstk
0,001000,1961-12-31,NaN,0.0,0.000,NaN,NaN,NaN
1,001000,1962-12-31,NaN,0.0,NaN,NaN,NaN,0.0
2,001000,1963-12-31,NaN,0.0,0.008,0.0,0.553,0.0
3,001000,1964-12-31,1.416,0.0,0.020,0.0,0.607,0.0
4,001000,1965-12-31,2.310,0.0,0.000,0.0,0.491,0.0
...,...,...,...,...,...,...,...,...
65167,355398,2021-12-31,NaN,NaN,NaN,NaN,NaN,NaN
65168,355398,2022-12-31,576.112,0.0,3.611,0.0,239.806,0.0
65169,355398,2023-12-31,576.883,0.0,1.347,0.0,270.086,0.0
65170,356128,2022-12-31,11072.847,0.0,7.016,0.0,1771.010,0.0


In [5]:
comp['year'] = comp['datadate'].dt.year

In [6]:
# 우선주(preferred stock) 값을 계산
# 'pstkrv'가 null인 경우 'pstkl'을 사용
# 'pstkl'도 null인 경우 'pstk'를 사용
# 모두 null인 경우 0으로 설정
comp['ps'] = np.where(comp['pstkrv'].isnull(), comp['pstkl'], comp['pstkrv'])
comp['ps'] = np.where(comp['ps'].isnull(), comp['pstk'], comp['ps'])
comp['ps'] = np.where(comp['ps'].isnull(), 0, comp['ps'])

In [7]:
# 'txditc' 열의 null 값을 0으로 대체
comp['txditc'] = comp['txditc'].fillna(0)

In [8]:
# 자기자본(book equity) 값을 계산
comp['be'] = comp['seq'] + comp['txditc'] - comp['ps']
# 계산된 자기자본 값이 0보다 큰 경우 그대로 사용하고, 그렇지 않은 경우 null로 설정
comp['be'] = np.where(comp['be'] > 0, comp['be'], np.nan)

In [9]:
# Compustat에서 연도 수를 계산
# 'gvkey'와 'datadate'를 기준으로 데이터를 정렬
comp = comp.sort_values(by=['gvkey', 'datadate'])
# 각 'gvkey' 그룹별로 행의 순서를 계산하여 'count' 열에 저장
comp['count'] = comp.groupby(['gvkey']).cumcount()

In [10]:
# 최종적으로 필요한 열들만 선택하여 comp 데이터프레임을 구성
comp = comp[['gvkey', 'datadate', 'year', 'be', 'count']]

# CRSP

In [16]:
crsp_m = conn.raw_sql("""
                      select a.permno, a.permco, a.date, b.shrcd, b.exchcd,
                      a.ret, a.retx, a.shrout, a.prc
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '01/01/1959' and '12/31/2017'
                      and b.exchcd between 1 and 3
                      """, date_cols=['date']) 

ProgrammingError: (psycopg2.errors.UndefinedColumn) column b.gvkey does not exist
LINE 2:                       select b.gvkey, a.permno, a.permco, a....
                                     ^

[SQL: 
                      select b.gvkey, a.permno, a.permco, a.date, b.shrcd, b.exchcd,
                      a.ret, a.retx, a.shrout, a.prc
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '01/01/1959' and '12/31/2017'
                      and b.exchcd between 1 and 3
                      ]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [12]:
# 특정 열들의 데이터 타입을 정수형으로 변경
crsp_m[['permco', 'permno', 'shrcd', 'exchcd']] = crsp_m[['permco', 'permno', 'shrcd', 'exchcd']].astype(int)

# 날짜를 월말로 정렬
crsp_m['jdate'] = crsp_m['date'] + MonthEnd(0)

# 상장폐지 수익률을 추가하기 위해 데이터베이스에서 상장폐지 데이터를 가져옴
dlret = conn.raw_sql("""
                     select permno, dlret, dlstdt 
                     from crsp.msedelist
                     """, date_cols=['dlstdt'])

# 상장폐지 데이터의 permno 열을 정수형으로 변경
dlret.permno = dlret.permno.astype(int)
# 상장폐지 날짜를 월말로 정렬
dlret['jdate'] = dlret['dlstdt'] + MonthEnd(0)

# crsp_m과 상장폐지 데이터를 permno와 jdate를 기준으로 병합
crsp = pd.merge(crsp_m, dlret, how='left', on=['permno', 'jdate'])
# 상장폐지 수익률(dlret)의 결측값을 0으로 대체
crsp['dlret'] = crsp['dlret'].fillna(0)
# 수익률(ret)의 결측값을 0으로 대체
crsp['ret'] = crsp['ret'].fillna(0)

# 상장폐지 수익률을 반영한 조정 수익률(retadj)을 계산
crsp['retadj'] = (1 + crsp['ret']) * (1 + crsp['dlret']) - 1

# 시가총액(market equity)을 계산
crsp['me'] = crsp['prc'].abs() * crsp['shrout'] 

# 필요 없는 열들(dlret, dlstdt, prc, shrout)을 삭제
crsp = crsp.drop(['dlret', 'dlstdt', 'prc', 'shrout'], axis=1)

# jdate, permco, me를 기준으로 데이터 정렬
crsp = crsp.sort_values(by=['jdate', 'permco', 'me'])

In [13]:
### Aggregate Market Cap ###
# 같은 permco에 속하는 다른 permno의 시가총액(me)을 합산하여 날짜별로 그룹화
crsp_summe = crsp.groupby(['jdate', 'permco'])['me'].sum().reset_index()

# 같은 permco와 날짜 내에서 가장 큰 시가총액을 가진 행 선택
crsp_maxme = crsp.groupby(['jdate', 'permco'])['me'].max().reset_index()

# jdate와 maxme를 기준으로 병합하여 해당 permno를 찾음
crsp1 = pd.merge(crsp, crsp_maxme, how='inner', on=['jdate', 'permco', 'me'])

# 'me' 열을 삭제하고 합산된 시가총액으로 대체
crsp1 = crsp1.drop(['me'], axis=1)

# 합산된 시가총액 정보와 병합하여 올바른 시가총액 정보 가져오기
crsp2 = pd.merge(crsp1, crsp_summe, how='inner', on=['jdate', 'permco'])

# permno와 날짜별로 정렬하고 중복 제거
crsp2 = crsp2.sort_values(by=['permno', 'jdate']).drop_duplicates()

# 12월의 시가총액만 유지
crsp2['year'] = crsp2['jdate'].dt.year
crsp2['month'] = crsp2['jdate'].dt.month
decme = crsp2[crsp2['month'] == 12]
decme = decme[['permno', 'date', 'jdate', 'me', 'year']].rename(columns={'me': 'dec_me'})

### July to June dates
# jdate 기준으로 6개월 전 날짜를 계산하여 ffdate로 설정
crsp2['ffdate'] = crsp2['jdate'] + MonthEnd(-6)
# ffdate를 기준으로 연도와 월을 계산하여 새로운 열 생성
crsp2['ffyear'] = crsp2['ffdate'].dt.year
crsp2['ffmonth'] = crsp2['ffdate'].dt.month
crsp2['1+retx'] = 1 + crsp2['retx']
crsp2 = crsp2.sort_values(by=['permno', 'date'])

# 주식별 누적 수익률 계산
crsp2['cumretx'] = crsp2.groupby(['permno', 'ffyear'])['1+retx'].cumprod()

# 누적 수익률의 이전 값을 계산하여 lcumretx로 설정
crsp2['lcumretx'] = crsp2.groupby(['permno'])['cumretx'].shift(1)

# 시가총액의 이전 값을 계산하여 lme로 설정
crsp2['lme'] = crsp2.groupby(['permno'])['me'].shift(1)

# 첫 permno인 경우 누락된 값을 대체하기 위해 me/(1+retx) 사용
crsp2['count'] = crsp2.groupby(['permno']).cumcount()
crsp2['lme'] = np.where(crsp2['count'] == 0, crsp2['me'] / crsp2['1+retx'], crsp2['lme'])

# 기본 시가총액을 mebase로 설정
mebase = crsp2[crsp2['ffmonth'] == 1][['permno', 'ffyear', 'lme']].rename(columns={'lme': 'mebase'})

# 결과를 다시 병합
crsp3 = pd.merge(crsp2, mebase, how='left', on=['permno', 'ffyear'])
crsp3['wt'] = np.where(crsp3['ffmonth'] == 1, crsp3['lme'], crsp3['mebase'] * crsp3['lcumretx'])

# 연도를 1년 증가시키고 필요한 열만 선택
decme['year'] = decme['year'] + 1
decme = decme[['permno', 'year', 'dec_me']]

# 6월 정보만 유지
crsp3_jun = crsp3[crsp3['month'] == 6]

# 6월 데이터를 연도별 12월 시가총액과 병합
crsp_jun = pd.merge(crsp3_jun, decme, how='inner', on=['permno', 'year'])
crsp_jun = crsp_jun[['permno', 'date', 'jdate', 'shrcd', 'exchcd', 'retadj', 'me', 'wt', 'cumretx', 'mebase', 'lme', 'dec_me']]
crsp_jun = crsp_jun.sort_values(by=['permno', 'jdate']).drop_duplicates()

In [14]:
# linkenddt 값이 누락된 경우 현재 날짜로 설정
link_table['linkenddt'] = link_table['linkenddt'].fillna(pd.to_datetime('today'))

# Compustat 데이터를 링크 테이블과 병합
ccm1 = pd.merge(comp[['gvkey', 'datadate', 'be', 'count']], link_table, how='left', on=['gvkey'])
ccm1['yearend'] = ccm1['datadate'] + YearEnd(0)
ccm1['jdate'] = ccm1['yearend'] + MonthEnd(6)

# 링크 날짜 범위 설정
ccm2 = ccm1[(ccm1['jdate'] >= ccm1['linkdt']) & (ccm1['jdate'] <= ccm1['linkenddt'])]
ccm2 = ccm2[['gvkey', 'permno', 'datadate', 'yearend', 'jdate', 'be', 'count']]

# Compustat과 CRSP 데이터를 permno와 jdate를 기준으로 병합
crsp_jun = pd.merge(crsp, ccm2, how='inner', on=['permno', 'jdate'])
crsp_jun['beme'] = crsp_jun['be'] * 1000 / crsp_jun['dec_me']

ProgrammingError: (psycopg2.errors.InsufficientPrivilege) permission denied for schema crsp_a_ccm

[SQL: 
                  select gvkey, lpermno as permno, linktype, linkprim, 
                  linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and (linkprim ='C' or linkprim='P')
                  ]
(Background on this error at: https://sqlalche.me/e/20/f405)